In [11]:
import torch

count=1
x_data=[[1,2],[2,3],[3,1],[4,3],[5,3],[6,2]]
y_data=[[0],[0],[0],[1],[1],[1]]
x_train=torch.FloatTensor(x_data)
y_train=torch.FloatTensor(y_data)

W=torch.zeros((2,1),requires_grad=True)
b=torch.zeros(1,requires_grad=True)

hypothesis=1/(1+torch.exp(-(x_train.matmul(W)+b)))
print("{}. e^1={}".format(count,torch.exp(torch.FloatTensor([1])))) # 1. e^x 자연상수
count+=1
hypothesis=torch.sigmoid(x_train.matmul(W)+b)
print("{}. 1/(1+e^-1)={}".format(count,torch.sigmoid(torch.FloatTensor([1])))) # 3. sigmoid 함수
count+=1
print("{}.\n {}".format(count,hypothesis)) # 3. y_train의 예측값 (1. 과 2. 방식 모두 결과가 동일함)
count+=1

cost=(-(y_train*torch.log(hypothesis)+(1-y_train)*torch.log(1-hypothesis))).mean()
cost=torch.nn.functional.binary_cross_entropy(hypothesis, y_train)
print("{}. {}".format(count,cost)) # 4. y_train과 예측값의 오차 (위의 두 방식 모두 결과가 동일함)
count+=1

1. e^1=tensor([2.7183])
2. 1/(1+e^-1)=tensor([0.7311])
3.
 tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward>)
4. 0.6931471824645996
5. 0.6931471824645996


In [20]:
import torch

x_data=[[1,2],[2,3],[3,1],[4,3],[5,3],[6,2]]
y_data=[[0],[0],[0],[1],[1],[1]]
x_train=torch.FloatTensor(x_data)
y_train=torch.FloatTensor(y_data)

W=torch.zeros((2,1),requires_grad=True)
b=torch.zeros(1,requires_grad=True)
optimizer=torch.optim.SGD([W,b],lr=1)
nb_epochs=1000
for epoch in range(nb_epochs+1):
    hypothesis=torch.sigmoid(x_train.matmul(W)+b)
    cost=torch.nn.functional.binary_cross_entropy(hypothesis, y_train) # cost=-(y_train*torch.log(hypothesis)+(1-y_train)*torch.log(1-hypothesis)).mean()

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 200==0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch,nb_epochs,cost.item()))
prediction=(hypothesis>=torch.FloatTensor([0.5])).int() # 0.5를 기준으로 학습된 결과를 구분함.
for i in range(len(prediction)):
    correct_prediction=prediction[i].item()==y_train[i]
    print("{}. correct: {} prediction: {} y_train: {}".format(
        i,correct_prediction.item(),prediction[i].item(),y_train[i]))

Epoch    0/1000 Cost: 0.693147
Epoch  200/1000 Cost: 0.080643
Epoch  400/1000 Cost: 0.045300
Epoch  600/1000 Cost: 0.031672
Epoch  800/1000 Cost: 0.024394
Epoch 1000/1000 Cost: 0.019852
0. correct: True prediction: 0 y_train: tensor([0.])
1. correct: True prediction: 0 y_train: tensor([0.])
2. correct: True prediction: 0 y_train: tensor([0.])
3. correct: True prediction: 1 y_train: tensor([1.])
4. correct: True prediction: 1 y_train: tensor([1.])
5. correct: True prediction: 1 y_train: tensor([1.])


In [22]:
import torch
import numpy as np

class BinaryClassifier(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear=torch.nn.Linear(8,1)
        self.sigmoid=torch.nn.Sigmoid()

    def forward(self,x):
        return self.sigmoid(self.linear(x))

xy=np.loadtxt("Classification_data.csv",delimiter=',',dtype=np.float32)
x_data=xy[:,0:-1]
y_data=xy[:,[-1]]
x_train=torch.FloatTensor(x_data)
y_train=torch.FloatTensor(y_data)

model=BinaryClassifier()
optimizer=torch.optim.SGD(model.parameters(),lr=1)

nb_epochs=100
for epoch in range(nb_epochs+1):
    hypothesis=model(x_train)
    cost=torch.nn.functional.binary_cross_entropy(hypothesis,y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch%20==0:
        prediction=hypothesis >= torch.FloatTensor([0.5])
        correct_prediction=prediction.float()==y_train
        accuracy=correct_prediction.sum().item() / len(correct_prediction)
        print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format(
            epoch,nb_epochs,cost.item(),accuracy * 100,
        ))

Epoch    0/100 Cost: 0.829956 Accuracy 34.26%
Epoch   20/100 Cost: 0.547120 Accuracy 71.15%
Epoch   40/100 Cost: 0.511064 Accuracy 76.81%
Epoch   60/100 Cost: 0.494840 Accuracy 76.81%
Epoch   80/100 Cost: 0.486423 Accuracy 76.94%
Epoch  100/100 Cost: 0.481636 Accuracy 76.55%
